## Import libraries

In [1]:

import pandas as pd
import clickhouse_connect
import os
import missingno as msno
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

# constants
CH_USER = os.getenv("CH_USER")
CH_PASS = os.getenv("CH_PASS")
CH_IP = os.getenv('CH_IP')

# from tools import create_db_table_from_df, pd_tools, spark_tools

root_path = "."
tmp_path = f'{root_path}/tmp'
path_data = f'{root_path}/data'
path_prod_data = f'{path_data}/prod_data'
path_new_data=f'{path_data}/new_data'
path_tmp_data=f'{path_data}/tmp_data'

ch_client = clickhouse_connect.get_client(host=CH_IP, port=8123, username=CH_USER, password=CH_PASS)

## Initialize Spark Session

In [4]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf, SQLContext

from pyspark.sql import SparkSession
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

import os
#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3"
    ,"com.clickhouse:clickhouse-jdbc:0.6.0-patch5"
    ,"com.clickhouse:clickhouse-http-client:0.6.0-patch5"
    ,"org.apache.httpcomponents.client5:httpclient5:5.3.1"
    ,"com.github.housepower:clickhouse-native-jdbc:2.7.1"
    ,"ai.catboost:catboost-spark_3.4_2.13:1.2.5"

]
ram = 30
cpu = 22*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.jars.packages", ",".join(packages))
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", CH_IP)
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", CH_USER)
         .config("spark.sql.catalog.clickhouse.password", CH_PASS)
         .config("spark.sql.catalog.clickhouse.database", "default")
        #  .config("spark.spark.clickhouse.write.compression.codec", "lz4")
        #  .config("spark.clickhouse.read.compression.codec", "lz4")
        #  .config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") l
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
        #  .config("spark.clickhouse.write.batchSize", "1000000")
         #.config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
        #  .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
        #  .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
        #  .config("spark.clickhouse.options.socket_timeout", "600000000")
        #  .config("spark.clickhouse.options.connection_timeout", "600000000")
        #  .config("spark.clickhouse.options.query_timeout", "600000000")         
         .config("spark.executor.memory", f"{ram}g")
        #  .config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
        #  .config("spark.driver.memory", f"{ram}g")
        #  .config("spark.executor.memoryOverhead", f"{ram}g")
        #  .config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )
#SedonaRegistrator.registerAll(spark)
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")


spark.sql("use clickhouse")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.github.housepower#clickhouse-spark-runtime-3.4_2.12 added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
com.github.housepower#clickhouse-native-jdbc added as a dependency
ai.catboost#catboost-spark_3.4_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-79b2865b-7edd-43f0-be23-a07af8a0b410;1.0
	confs: [default]
	found com.github.housepower#clickhouse-spark-runtime-3.4_2.12;0.7.3 in central
	found com.clickhouse#clickhouse-jdbc;0.6.0-patch5 in central
	found com.clickhouse#clickhouse-http-client;0.6.0-patch5 in central
	found com.clickhouse#clickhouse-client;0.6.0-patch5 in central
	found com.clickhouse#clickhouse-data;0.6.0-patch5 in central
	found org.apache.httpcomponents.client5#httpclient5;5.3

DataFrame[]

## Find active email users

In [5]:
spark.sql('select * from tmp.messages').show()

+----------+---------+-----------+----------------+
|message_id|sender_id|receiver_id|reply_message_id|
+----------+---------+-----------+----------------+
|         1|        1|          2|            null|
|         3|        2|          1|               1|
|        19|        3|          4|              16|
|        16|        4|          3|               1|
|         1|        3|          4|            null|
|         4|        1|          2|               3|
|         7|        3|          4|            null|
|        17|        4|          2|               7|
+----------+---------+-----------+----------------+



In [63]:
messages = pd.read_csv(f'{path_data}/messages.csv')
messages['reply_message_id'] = messages['reply_message_id'].astype('Int32')
messages

,message_id,sender_id,receiver_id,reply_message_id
0,1,1,2,<NA>
1,3,2,1,1
2,19,3,4,16
3,16,4,3,1
4,1,3,4,<NA>
5,4,1,2,3
6,7,3,4,<NA>
7,17,4,3,7


In [64]:
# upload data to clickhouse
ch_client.insert_df('messages', messages, 'tmp')


In [65]:
ch_client.query_df('''select * from tmp.messages''')

,message_id,sender_id,receiver_id,reply_message_id
0,1,1,2,<NA>
1,3,2,1,1
2,19,3,4,16
3,16,4,3,1
4,1,3,4,<NA>
5,4,1,2,3
6,7,3,4,<NA>
7,17,4,3,7


Пользователи 1 и 2 общались в рамках одного треда. который можно представить в виде
Цепочки из message id: 1 — 3 — 4. 

Пользователи 3 и 4 общались в рамках двух тредов,которые также можно представить в виде цепочек из message id: 1—16-19 и 7 — 17.

Вашей задачей является создание запроса на SQLite, который для каждого пользователя
посчитает максимальную длину треда, в котором он принимал участие (длиной треда является
количество сообщений в нем). 

Результатом должна быть таблица с двумя колонками: 
- user_id (идентификатор пользователя из sender_id/receiver_id) 
- max_thread_messages (максимальная длина треда для данного пользователя). 

Выходная таблица должна быть отсортирована по убыванию max_thread_messages (в случае равенства мах thread messages
предпочтение должно отдаваться пользователю с наименьшим user_id). 

Гарантируется, что одному сообщению может соответствовать только один ответ и для каждого непустого значения reply message id в таблице присутствует сообщение с
соответствующим message id в рамках определенной пары пользователей.

In [84]:
ch_client.query_df('''
    with
    -- group by sender_id = 1, 2 and message_id = 1, 3, 4
        send_messages_1 as (
            select
                case 
                    when t.message_id in (1, 3, 4) then 1
                end as thread_id,
                t.sender_id as user_id,
                count(t.sender_id) as send_messages
            from tmp.messages t
            where 
                t.message_id in (1, 3, 4) 
                and t.sender_id in (1, 2)
            group by t.sender_id, thread_id
        ),
    -- group by sender_id = 3, 4 and message_id = 1, 16, 19
        send_messages_2 as (
            select
                case 
                    when t.message_id in (1, 16, 19) then 2
                end as thread_id,
                t.sender_id as user_id,
                count(t.sender_id) as send_messages
            from tmp.messages t
            where 
                t.message_id in (1, 16, 19) 
                and t.sender_id in (3, 4)
            group by t.sender_id, thread_id
        ),
    -- group by sender_id = 3, 4 and message_id = 7, 17
        send_messages_3 as (
            select
                case
                    when t.message_id in (7, 17) then 3
                end as thread_id,
                t.sender_id as user_id,
                count(t.sender_id) as send_messages
            from tmp.messages t
            where
                t.message_id in (7, 17)
                and t.sender_id in (3, 4)
            group by t.sender_id, thread_id
        ),                                    
    -- group by receiver_id = 1, 2                
        receive_messages_1 as (
            select
                case 
                    when t.message_id in (1, 3, 4) then 1
                end as thread_id,
                t.receiver_id as user_id,
                count(t.receiver_id) as receive_messages
            from tmp.messages t
            where 
                t.message_id in (1, 3, 4) 
                and t.receiver_id in (1, 2)
            group by t.receiver_id, thread_id
        ),
    -- group by receiver_id = 3, 4 and message_id = 1, 16, 19
        receive_messages_2 as (
            select
                case 
                    when t.message_id in (1, 16, 19) then 2
                end as thread_id,
                t.receiver_id as user_id,
                count(t.receiver_id) as receive_messages
            from tmp.messages t
            where 
                t.message_id in (1, 16, 19) 
                and t.receiver_id in (3, 4)
            group by t.receiver_id, thread_id
        ),
    -- group by receiver_id = 3, 4 and message_id = 7, 17
        receive_messages_3 as (
            select 
                case 
                    when t.message_id in (7, 17) then 3
                end as thread_id,
                t.receiver_id as user_id,
                count(t.receiver_id) as receive_messages
            from tmp.messages t
            where 
                t.message_id in (7, 17) 
                and t.receiver_id in (3, 4)
            group by t.receiver_id, thread_id
        ),                   
    -- Combine send_messages_1, send_messages_2, and send_messages_3
        all_send_messages as (
            select * from send_messages_1
            union all
            select * from send_messages_2
            union all
            select * from send_messages_3
        ),
    -- Combine receive_messages_1, receive_messages_2, and receive_messages_3
        all_receive_messages as (
            select * from receive_messages_1
            union all
            select * from receive_messages_2
            union all
            select * from receive_messages_3
        ),
    -- Calculate max_thread_messages by user_id
        user_thread_messages as (
            select
                coalesce(s.user_id, r.user_id) as user_id,
                coalesce(s.thread_id, r.thread_id) as thread_id,
                coalesce(s.send_messages, 0) as send_messages,
                coalesce(r.receive_messages, 0) as receive_messages,
                coalesce(s.send_messages, 0) + coalesce(r.receive_messages, 0) as max_thread_messages 
            from all_send_messages s
            full outer join all_receive_messages r
            on s.user_id = r.user_id and s.thread_id = r.thread_id
        )
    -- Select maximum max_thread_messages by user_id                              
    select 
        u.user_id user_id,
        max(u.max_thread_messages) as max_thread_messages
    from user_thread_messages u
    group by user_id
    order by max_thread_messages desc, user_id        
''')


,user_id,max_thread_messages
0,1,3
1,2,3
2,3,3
3,4,3


## Coin toss up

У вас есть п монет. Любое количество из них вы можете сдать, получив за это соответствующее вознаграждение. Оставшиеся монеты подбрасываются, и вы получаете 3^k
рублей. где k — количество выпавших решек, Какова оптимальная стратегия?

**Формат ввода**
В первой строке целое положительное число n <= 60 — количество монет.
Во второй строке n целых неотрицательных чисел a1, a2,..., an, через пробел, где аi <= 10^18 — вознаграждение за возврат i монет.

**Формат вывода**
Выведите количество сдаваемых монет. при которых математическое ожидание суммарного выигрыша максимально. Если оптимальных стратегий несколько, выведите все варианты
через пробел в порядке возрастания.

**Пример1:**
n = 1
a = 5

Вывод: 1
В первом примере монета всего одна. в таком случае всего два варианта: сдавать монету или не сдавать. Сдав монету. мы получим гарантированные 6 рублей (5 за сдачу монеты и 1 = к за 0 выпавших решек, так как не было монет для подбрасывания); не сдав монету, мы получим 1 рубль или 3 рубля в зависимости от результата подбрасывания. Видно, что оптимальным вариантом является сдать монету